In [36]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import gc
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import ols
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

pd.set_option('display.max_rows', 200)
pd.set_option("display.max_columns", 200)

In [38]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
sample_sub = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/sample_submission.csv'
bureau_data = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/bureau.csv'
bureau_balance = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/bureau_balance.csv'
pc_balance = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/POS_CASH_balance.csv'
test = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/application_test.csv'
train = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/application_train.csv'
cc_balance = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/credit_card_balance.csv'
install_balance = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/installments_payments.csv'
prev_app = '/content/drive/MyDrive/Project DS/Home-Credit-Indonesia/final project/dataset/previous_application.csv'

# Functions

In [40]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [41]:
def data_nan(dataframe):
  print('The shape:', dataframe.shape)
  result = pd.concat([dataframe.isnull().sum().reset_index(name ='sum_of_nulls'), 
                      dataframe.dtypes.reset_index(name="dtypes")], axis=1)\
                      .T.drop_duplicates()\
                      .T.sort_values('sum_of_nulls', ascending=False)
  return result

In [42]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]

    del original_columns; gc.collect()
    return df, categorical_columns

In [43]:
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [44]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

In [45]:
# Korelasyonlar
def high_correlation(data, remove=['SK_ID_CURR', 'SK_ID_BUREAU'], corr_coef="pearson", corr_value = 0.7):
    if len(remove) > 0:
        cols = [x for x in data.columns if (x not in remove)]
        c = data[cols].corr(method=corr_coef)
    else:
        c = data.corr(method=corr_coef)

    for i in c.columns:
        cr = c.loc[i].loc[(c.loc[i] >= corr_value) | (c.loc[i] <= -corr_value)].drop(i)
        if len(cr) > 0:
            print(i)
            print("-------------------------------")
            print(cr.sort_values(ascending=False))
            print("\n")

In [46]:
# Rare Encoding
def rare_encoder(dataframe, rare_perc, cat_cols):
   
    rare_columns = [col for col in cat_cols if
                    (dataframe[col].value_counts() / len(dataframe) < rare_perc).sum() > 1]

    for col in rare_columns:
        tmp = dataframe[col].value_counts() / len(dataframe)
        rare_labels = tmp[tmp < rare_perc].index
        dataframe[col] = np.where(dataframe[col].isin(rare_labels), 'Rare', dataframe[col])

    return dataframe

# Data Processing Bureu

In [47]:
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(path, nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    
    # Calculate rate for each category with decay
    bb_processed_cat = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    bb_processed_num = bb.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].size().reset_index()
    
    bb_processed = bb_processed_cat.merge(bb_processed_num, how='left', on='SK_ID_BUREAU')

    bb_processed.rename(columns = {'MONTHS_BALANCE':'MONTHS_BALANCE_SIZE_BUREAU_BALANCE'}, inplace = True)
    # Min, Max, Count and mean duration of payments (months)
    #agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
   # bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

In [48]:
def get_bureau(path, num_rows= None, cat_cols = False):
    """ Process bureau.csv and bureau_balance.csv and return a pandas dataframe. """
    bureau = pd.read_csv(path, nrows= num_rows)
  
    # Handling NaN
    bureau = bureau.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
    bureau = bureau.apply(lambda x: x.fillna(x.mode()[0]) if x.dtype == "O" else x, axis=0)

    # Credit duration and credit/account end date difference
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    
    # Credit to debt ratio and difference
    bureau['CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
   # bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['ANNUITY_TO_CREDIT_RATIO'] =   bureau['AMT_ANNUITY'] / bureau['AMT_CREDIT_SUM']

    # Fillna to zero
    bureau.fillna(0, inplace=True)
    # One-hot encoder
    bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category= False)

    #bb_process = bureau.groupby('SK_ID_CURR')[categorical_cols].mean().reset_index()
    # Join bureau balance features
    #bureau = bureau.merge(get_bureau_balance(bureau_balance, num_rows), how='left', on='SK_ID_BUREAU')

    if cat_cols == True:
      return bureau, categorical_cols

    del categorical_cols; gc.collect()
    
    return bureau


In [49]:
def pipeline_bureau():
  bureau, cols = get_bureau(bureau_data, cat_cols=True)
  bb = get_bureau_balance(bureau_balance)

  # Join bureau balance features
  result = bureau.merge(bb, how='left', on='SK_ID_BUREAU').fillna(0)
  #bb_process = result.groupby('SK_ID_CURR')[cols].mean().reset_index()
  metrics = ["CREDIT_DURATION","ENDDATE_DIF", 'CREDIT_DEBT_RATIO', 'ANNUITY_TO_CREDIT_RATIO']
  columns = list(cols) + bb.columns.tolist() +  metrics + ["SK_ID_CURR"] 
  result = result[columns]
  result = pd.pivot_table(result, index='SK_ID_CURR').reset_index()

  del bureau, cols, bb, columns, metrics; gc.collect()

  return result

# Data Processing Credit Card Balance

In [50]:
def pipeline_credit_balance(num_rows=None):
  cc = pd.read_csv(cc_balance, nrows = num_rows)
  cc = cc[cc["AMT_DRAWINGS_ATM_CURRENT"] > 0]
  # Handling NaN
  amt_col = ["AMT_PAYMENT_CURRENT", "AMT_DRAWINGS_ATM_CURRENT", "AMT_INST_MIN_REGULARITY",
             "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT"]
  for col in amt_col:
    cc[col] = cc[col].fillna(cc[col].median())
    cc[col + "_TO_AMT_BALANCE"] = cc[col] / cc["AMT_BALANCE"] # Creating new metrics
  cc.fillna(0, inplace=True)

  drop_col = ["MONTHS_BALANCE", 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_CURRENT',
              'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', "AMT_RECIVABLE",
              'AMT_TOTAL_RECEIVABLE']
  drop_col = drop_col + amt_col
  cc.drop(drop_col, axis=1, inplace=True)
  
  # OneHotEncoding
  cc, categorical_cols = one_hot_encoder(cc, nan_as_category= False)

  # Aggregation
  cnt_col = ['CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
             'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM']

  # Active status
  active = cc[cc['NAME_CONTRACT_STATUS_Active'] == 1].drop("SK_ID_PREV", axis=1)
  active_agg = active.groupby('SK_ID_CURR').mean()
  active_agg.columns = pd.Index(['ACTIVE_' + e.upper() for e in active_agg.columns.tolist()])
  for col in ['ACTIVE_' + e.upper() for e in cnt_col]:
    active_agg[col] = active_agg[col].apply(lambda x: round(x))
  # Completed status
  completed = cc[cc['NAME_CONTRACT_STATUS_Completed'] == 1].drop("SK_ID_PREV", axis=1)
  completed_agg = completed.groupby('SK_ID_CURR').mean()
  completed_agg.columns = pd.Index(['COMPLETED_' + e.upper() for e in completed_agg.columns.tolist()])
  for col in ['COMPLETED_' + e.upper() for e in cnt_col]:
    completed_agg[col] = completed_agg[col].apply(lambda x: round(x))
  result = completed_agg.join(active_agg, how='left', on='SK_ID_CURR')
  # Signed status
  signed = cc[cc['NAME_CONTRACT_STATUS_Signed'] == 1].drop("SK_ID_PREV", axis=1)
  signed_agg = signed.groupby('SK_ID_CURR').mean()
  signed_agg.columns = pd.Index(['SIGNED_' + e.upper() for e in signed_agg.columns.tolist()])
  for col in ['SIGNED_' + e.upper() for e in cnt_col]:
    signed_agg[col] = signed_agg[col].apply(lambda x: round(x))  

  result = result.join(signed_agg, how='left', on='SK_ID_CURR').reset_index()

  result = result.apply(lambda x: x.replace(np.inf, np.nan) if x.dtype != "O" else x, axis=0)

  del amt_col, categorical_cols, active, active_agg, completed, completed_agg, signed, signed_agg; gc.collect()
  return result

# Installment Payment

In [51]:
def pipeline_installment_payment(num_rows=None):
  ip = pd.read_csv(install_balance, nrows=num_rows)

  # Handling NaN
  ip = ip.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)

  # Create new metrics
  ip['LATE_PAYMENT'] = -ip['DAYS_ENTRY_PAYMENT'] + ip['DAYS_INSTALMENT']
  ip['AMT_PAYMENT_TO_INSTALMENT'] =   ip['AMT_PAYMENT'] / ip['AMT_INSTALMENT']

  # Drop columns
  drop_cols = ["DAYS_INSTALMENT", "DAYS_ENTRY_PAYMENT", "AMT_INSTALMENT", "AMT_PAYMENT"]
  ip.drop(drop_cols, axis=1, inplace=True)

  # Aggregation
  result = ip.drop('SK_ID_PREV', axis=1).groupby('SK_ID_CURR').mean()
  for col in ['NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER']:
    result[col] = result[col].apply(lambda x: round(x))
  result = result.reset_index()
  del ip; gc.collect()
  return result
  

In [52]:
def pipeline_posh_cash_balance(num_rows=None):
  pc = pd.read_csv(pc_balance, nrows=num_rows)

  # Handling NaN
  pc = pc.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
  
  # OneHotEncoding
  pc, categorical_cols = one_hot_encoder(pc, nan_as_category= False)

  # Calculate months_balance 
#  pc = pc.groupby(['SK_ID_PREV', "SK_ID_CURR"])['MONTHS_BALANCE'].size().reset_index().\
#  merge(pc[["SK_ID_PREV", 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', "SK_DPD", "SK_DPD_DEF"]], how='left', on='SK_ID_PREV')

#  pc.rename(columns = {'MONTHS_BALANCE':'MONTHS_BALANCE_SIZE_POSH_CASH'}, inplace = True)
  # Aggregation
  pc_size = pc.drop('SK_ID_PREV', axis=1).groupby('SK_ID_CURR')['MONTHS_BALANCE'].size().reset_index()
  pc_sum = pc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1).groupby('SK_ID_CURR').mean().reset_index()

  pc = pc_size.merge(pc_sum, how='left', on='SK_ID_CURR')

  del pc_size, pc_sum; gc.collect()
  return pc



# Previous Applications

In [53]:
def pipeline_previous_app(num_rows=None):
  pa = pd.read_csv(prev_app, nrows=num_rows)

  # Handling 365.243 values to NaN
  pa['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
  pa['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
  pa['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
  pa['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
  pa['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

  pa = pa.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
  pa = pa.apply(lambda x: x.fillna(x.mode()) if x.dtype == "O" else x, axis=0)
  
  # Create new metrics
  pa['PREV_APPLICATION_CREDIT_RATIO'] = pa['AMT_APPLICATION'] / pa['AMT_CREDIT']
  pa['PREV_APPLICATION_ANNUITY_TO_CREDIT'] = pa['AMT_ANNUITY'] / pa['AMT_CREDIT']
  pa['PREV_APPLICATION_DOWN_PAYMENT_TO_CREDIT'] = pa['AMT_DOWN_PAYMENT'] / pa['AMT_CREDIT']

  # Interest ratio on previous application (simplified)
  total_payment = pa['AMT_ANNUITY'] * pa['CNT_PAYMENT']
  pa['SIMPLE_INTERESTS'] = (total_payment/pa['AMT_CREDIT'] - 1)/pa['CNT_PAYMENT']
  new_metrics = ['PREV_APPLICATION_CREDIT_RATIO', 'PREV_APPLICATION_ANNUITY_TO_CREDIT', 
                 'PREV_APPLICATION_DOWN_PAYMENT_TO_CREDIT','SIMPLE_INTERESTS']
  # Handling NaN categorical type
  pa = pa.apply(lambda x: x.fillna(x.mode()) if x.dtype == "O" else x, axis=0)
  drop_cols = ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_FIRST_DRAWING',
                'NAME_CASH_LOAN_PURPOSE', 'CODE_REJECT_REASON', 'FLAG_LAST_APPL_PER_CONTRACT',
                'NFLAG_LAST_APPL_IN_DAY', 'SELLERPLACE_AREA']
  pa.drop(drop_cols, axis=1, inplace=True)
  # One-hot encoding
  ohe_columns = [
        'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'CHANNEL_TYPE',
        'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
        'NAME_PRODUCT_TYPE', 'NAME_CLIENT_TYPE']

  pa =pa[["SK_ID_CURR","SK_ID_PREV"] + new_metrics + ohe_columns]

  pa, cat_cols = one_hot_encoder(pa, ohe_columns, nan_as_category= False)

  # Aggregation
  col_list = pa.columns.tolist()
  id_list = ["SK_ID_CURR","SK_ID_PREV"]
  num_list = [col for col in col_list if col not in cat_cols + id_list]
                 
  #pa = pa[new_metrics + cat_cols + id_list]
  prev_agg = pa.drop("SK_ID_PREV", axis=1).groupby('SK_ID_CURR').mean()
  prev_agg.columns = pd.Index(['PREV_' + e.upper() for e in prev_agg.columns.tolist()])
  # Previous Applications: Approved Applications - only numerical feature
  approved = pa[pa['NAME_CONTRACT_STATUS_Approved'] == 1].drop("SK_ID_PREV", axis=1)
  approved_agg = approved.groupby('SK_ID_CURR')[num_list].mean()
  approved_agg.columns = pd.Index(['APPROVED_' + e.upper() for e in approved_agg.columns.tolist()])
  prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
  # Previous Applications: Refused Applications - only numerical features
  refused = pa[pa['NAME_CONTRACT_STATUS_Refused'] == 1].drop("SK_ID_PREV", axis=1)
  refused_agg = refused.groupby('SK_ID_CURR')[num_list].mean()
  refused_agg.columns = pd.Index(['REFUSED_' + e.upper() for e in refused_agg.columns.tolist()])
  prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR').reset_index()

  prev_agg.fillna(0, inplace=True)
  del drop_cols, cat_cols, col_list, id_list, num_list, approved, approved_agg; gc.collect()
  #prev_agg = reduce_mem_usage(prev_agg)
  return prev_agg

# Application

In [54]:
def application_train_test(num_rows = None, nan_as_category = False):
    df = pd.read_csv(train, nrows= num_rows)
    test_df = pd.read_csv(test, nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()

    # Split type columns
    cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols = [x for x in num_cols if x !='TARGET']

    # Handling anomaly
    df = df[df['CODE_GENDER'] != 'XNA']
    df = df[df['NAME_FAMILY_STATUS'] != "Unknown" ]
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

    # Handling NaN
    df[num_cols] = df[num_cols].apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
    df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(x.mode()) if x.dtype == "O" else x, axis=0)
     
    # NAME_HOUSING_TYPE
    df["NAME_HOUSING_TYPE"] = np.where(df["NAME_HOUSING_TYPE"].str.contains("House / apartment"),
                                       "House_apartment", df["NAME_HOUSING_TYPE"])

    # FEATURE ENGINEERING
    df['NEW_DAYS_EMPLOYED_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['NEW_INCOME_PER_RATIO'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['NEW_ANNUITY_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['NEW_PAYMENT_RATIO'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    df["NEW_EXTSOURCE_MEAN"] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df["NEW_GOODS_CREDIT_RATIO"] = df["AMT_GOODS_PRICE"] / df["AMT_CREDIT"]
    df["NEW_GOODS_CREDIT_DIFF_RATIO"] = (df["AMT_GOODS_PRICE"] - df["AMT_CREDIT"]) / df["AMT_INCOME_TOTAL"]
    

    new_metrics = ['NEW_DAYS_EMPLOYED_RATIO', 'NEW_INCOME_CREDIT_RATIO', 'NEW_INCOME_PER_RATIO',
                   'NEW_ANNUITY_INCOME_RATIO', 'NEW_PAYMENT_RATIO', "NEW_EXTSOURCE_MEAN",
                   "NEW_GOODS_CREDIT_RATIO", "NEW_GOODS_CREDIT_DIFF_RATIO"]
    cat_metrics = ["NAME_HOUSING_TYPE"]
    id_target = ['TARGET', 'SK_ID_CURR']

    # One-Hot Encoding
    df = df[new_metrics + cat_metrics + id_target ]
    df, cat_cols = one_hot_encoder(df, nan_as_category=False)
    
    del test_df; gc.collect()
    df = df.reset_index(drop=True)
    return df

# Saving to feather

In [55]:
df = application_train_test()
df.to_feather('application_feather')
print(df.shape)
del df
df = pipeline_bureau()
df.to_feather('bureau_feather')
del df
df = pipeline_credit_balance()
df.to_feather('credit_balance_feather')
del df
df = pipeline_posh_cash_balance()
df.to_feather('posh_cash_balance_feather')
del df
df = pipeline_previous_app()
df.to_feather('previous_app_feather')
del df
df = pipeline_installment_payment()
df.to_feather('installment_payment_feather')
del df

Train samples: 307511, test samples: 48744
(356249, 16)


# Modeling

In [56]:
df = pd.read_feather('application_feather')
pos = pd.read_feather('posh_cash_balance_feather')
bb = pd.read_feather('bureau_feather')
cc = pd.read_feather('credit_balance_feather')
ins = pd.read_feather('installment_payment_feather')
prev = pd.read_feather('previous_app_feather')

print(df.shape, pos.shape, bb.shape, cc.shape, ins.shape, prev.shape)

for i in [pos, bb, cc, ins, prev]:
    df = pd.merge(df, i , how = "left", on = "SK_ID_CURR")
    
print(df.shape)

del pos, bb, ins, cc, prev

(356249, 16) (337252, 15) (305811, 38) (6, 46) (339587, 5) (338857, 65)
(356249, 180)


In [57]:
df = df.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
df = df.apply(lambda x: x.fillna(x.mode()) if x.dtype == "O" else x, axis=0)

In [58]:
# Train-Test Split
df.columns = list(map(lambda x: str(x).replace(" ", "_").replace("-", "_").replace("_/_", "_").upper(), df.columns))
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)

df.replace([np.inf, -np.inf], 0, inplace=True)
train = df[df.TARGET.isnull() == False]
test = df[df.TARGET.isnull()]

random_state_val =42
test_size_val =0.1
train, val = train_test_split(train, test_size = test_size_val, random_state = random_state_val)


x_train = train.drop(["TARGET", "SK_ID_CURR"], axis = 1)
x_train = x_train.fillna(0)
x_val = val.drop(["TARGET", "SK_ID_CURR"], axis = 1)
x_val = x_val.fillna(0)
x_test = test.drop(["TARGET", "SK_ID_CURR"], axis = 1)
x_test = x_test.fillna(0)
y_train = train.TARGET
y_val = val.TARGET

(356249, 180)


In [59]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
x_train, y_train = oversample.fit_resample(x_train, y_train)
x_val, y_val = oversample.fit_resample(x_val, y_val)

In [61]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)


LogisticRegression(max_iter=1000)

In [65]:
from sklearn.metrics import confusion_matrix

fpr, tpr, _ = roc_curve(y_train, model.predict_proba(x_train.values)[:, 1])
roc_auc = auc(fpr, tpr)

pred_val = model.predict(x_train)
tp, fn, fp, tn = confusion_matrix(y_train, pred_val, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(y_train.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(y_train.values.ravel(), pred_val))

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("3. Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

1. Counfusion Matrix
                   True Value 0  True Value 1
Predicted Value 0        193341        152562
Predicted Value 1        104976        145755

2. Classification Report
              precision    recall  f1-score   support

         0.0       0.56      0.65      0.60    298317
         1.0       0.58      0.49      0.53    298317

    accuracy                           0.57    596634
   macro avg       0.57      0.57      0.57    596634
weighted avg       0.57      0.57      0.57    596634

3. Model Metric Sumamry
 - Accuracy Rate    : 56.835 %
 - Recall Rate      : 48.859 %
 - Precision Rate   : 58.132 %
 - Specificity Rate : 64.811 %
 - F1 Score         : 53.094 
 - ROC AUC          : 59.745 


In [64]:
from sklearn.metrics import confusion_matrix

fpr, tpr, _ = roc_curve(y_val, model.predict_proba(x_val.values)[:, 1])
roc_auc = auc(fpr, tpr)

pred_val = model.predict(x_val)
tp, fn, fp, tn = confusion_matrix(y_val, pred_val, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(y_val.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(y_val.values.ravel(), pred_val))

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("3. Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

1. Counfusion Matrix
                   True Value 0  True Value 1
Predicted Value 0         21254         17620
Predicted Value 1         11853         15487

2. Classification Report
              precision    recall  f1-score   support

         0.0       0.55      0.64      0.59     33107
         1.0       0.57      0.47      0.51     33107

    accuracy                           0.55     66214
   macro avg       0.56      0.55      0.55     66214
weighted avg       0.56      0.55      0.55     66214

3. Model Metric Sumamry
 - Accuracy Rate    : 55.488 %
 - Recall Rate      : 46.779 %
 - Precision Rate   : 56.646 %
 - Specificity Rate : 64.198 %
 - F1 Score         : 51.242 
 - ROC AUC          : 58.286 


In [75]:
# Model
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    nthread=4,
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=34,
    colsample_bytree=0.9497036,
    subsample=0.8715623,
    max_depth=8,
    reg_alpha=0.041545473,
    reg_lambda=0.0735294,
    min_split_gain=0.0222415,
    min_child_weight=39.3259775,
    silent=-1,
    verbose=-1, )

model.fit(x_train, y_train, eval_set=[(x_val, y_val)],
        eval_metric='auc', verbose=200)
        

[200]	valid_0's auc: 0.766106	valid_0's binary_logloss: 0.579374
[400]	valid_0's auc: 0.773475	valid_0's binary_logloss: 0.571033
[600]	valid_0's auc: 0.77582	valid_0's binary_logloss: 0.568782
[800]	valid_0's auc: 0.776755	valid_0's binary_logloss: 0.568218
[1000]	valid_0's auc: 0.77688	valid_0's binary_logloss: 0.568471
[1200]	valid_0's auc: 0.77688	valid_0's binary_logloss: 0.569013
[1400]	valid_0's auc: 0.776445	valid_0's binary_logloss: 0.570151
[1600]	valid_0's auc: 0.776078	valid_0's binary_logloss: 0.571322
[1800]	valid_0's auc: 0.775667	valid_0's binary_logloss: 0.572618
[2000]	valid_0's auc: 0.7752	valid_0's binary_logloss: 0.574074
[2200]	valid_0's auc: 0.774992	valid_0's binary_logloss: 0.575481
[2400]	valid_0's auc: 0.774617	valid_0's binary_logloss: 0.57701
[2600]	valid_0's auc: 0.774287	valid_0's binary_logloss: 0.578502
[2800]	valid_0's auc: 0.774377	valid_0's binary_logloss: 0.579761
[3000]	valid_0's auc: 0.773727	valid_0's binary_logloss: 0.58178
[3200]	valid_0's auc:

LGBMClassifier(colsample_bytree=0.9497036, learning_rate=0.02, max_depth=8,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               n_estimators=10000, nthread=4, num_leaves=34,
               reg_alpha=0.041545473, reg_lambda=0.0735294, silent=-1,
               subsample=0.8715623, verbose=-1)

In [76]:
from sklearn.metrics import confusion_matrix

fpr, tpr, _ = roc_curve(y_train, model.predict_proba(x_train.values)[:, 1])
roc_auc = auc(fpr, tpr)

pred_val = model.predict(x_train)
tp, fn, fp, tn = confusion_matrix(y_train, pred_val, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(y_train.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(y_train.values.ravel(), pred_val))

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("3. Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

1. Counfusion Matrix
                   True Value 0  True Value 1
Predicted Value 0        271476          8471
Predicted Value 1         26841        289846

2. Classification Report
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94    298317
         1.0       0.92      0.97      0.94    298317

    accuracy                           0.94    596634
   macro avg       0.94      0.94      0.94    596634
weighted avg       0.94      0.94      0.94    596634

3. Model Metric Sumamry
 - Accuracy Rate    : 94.081 %
 - Recall Rate      : 97.160 %
 - Precision Rate   : 91.524 %
 - Specificity Rate : 91.003 %
 - F1 Score         : 94.258 
 - ROC AUC          : 98.621 


In [78]:
from sklearn.metrics import confusion_matrix

fpr, tpr, _ = roc_curve(y_val, model.predict_proba(x_val.values)[:, 1])
roc_auc = auc(fpr, tpr)

pred_val = model.predict(x_val)
tp, fn, fp, tn = confusion_matrix(y_val, pred_val, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(y_val.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(y_val.values.ravel(), pred_val))

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("3. Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

1. Counfusion Matrix
                   True Value 0  True Value 1
Predicted Value 0         29083         18983
Predicted Value 1          4024         14124

2. Classification Report
              precision    recall  f1-score   support

         0.0       0.61      0.88      0.72     33107
         1.0       0.78      0.43      0.55     33107

    accuracy                           0.65     66214
   macro avg       0.69      0.65      0.63     66214
weighted avg       0.69      0.65      0.63     66214

3. Model Metric Sumamry
 - Accuracy Rate    : 65.254 %
 - Recall Rate      : 42.662 %
 - Precision Rate   : 77.827 %
 - Specificity Rate : 87.845 %
 - F1 Score         : 55.113 
 - ROC AUC          : 75.764 


# Save best model

In [70]:
# Model
from lightgbm import LGBMClassifier

# LightGBM parameters found by Bayesian optimization
clf = LGBMClassifier(
    nthread=4,
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=34,
    colsample_bytree=0.9497036,
    subsample=0.8715623,
    max_depth=8,
    reg_alpha=0.041545473,
    reg_lambda=0.0735294,
    min_split_gain=0.0222415,
    min_child_weight=39.3259775,
    silent=-1,
    verbose=-1, )

clf.fit(x_train, y_train, eval_set=[(x_val, y_val)],
        eval_metric='auc', verbose=200)

[200]	valid_0's auc: 0.766106	valid_0's binary_logloss: 0.579374
[400]	valid_0's auc: 0.773475	valid_0's binary_logloss: 0.571033
[600]	valid_0's auc: 0.77582	valid_0's binary_logloss: 0.568782
[800]	valid_0's auc: 0.776755	valid_0's binary_logloss: 0.568218
[1000]	valid_0's auc: 0.77688	valid_0's binary_logloss: 0.568471
[1200]	valid_0's auc: 0.77688	valid_0's binary_logloss: 0.569013
[1400]	valid_0's auc: 0.776445	valid_0's binary_logloss: 0.570151
[1600]	valid_0's auc: 0.776078	valid_0's binary_logloss: 0.571322
[1800]	valid_0's auc: 0.775667	valid_0's binary_logloss: 0.572618
[2000]	valid_0's auc: 0.7752	valid_0's binary_logloss: 0.574074
[2200]	valid_0's auc: 0.774992	valid_0's binary_logloss: 0.575481
[2400]	valid_0's auc: 0.774617	valid_0's binary_logloss: 0.57701
[2600]	valid_0's auc: 0.774287	valid_0's binary_logloss: 0.578502
[2800]	valid_0's auc: 0.774377	valid_0's binary_logloss: 0.579761
[3000]	valid_0's auc: 0.773727	valid_0's binary_logloss: 0.58178
[3200]	valid_0's auc:

LGBMClassifier(colsample_bytree=0.9497036, learning_rate=0.02, max_depth=8,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               n_estimators=10000, nthread=4, num_leaves=34,
               reg_alpha=0.041545473, reg_lambda=0.0735294, silent=-1,
               subsample=0.8715623, verbose=-1)

LGBMClassifier(colsample_bytree=0.9497036, learning_rate=0.02, max_depth=8,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               n_estimators=10000, nthread=4, num_leaves=34,
               reg_alpha=0.041545473, reg_lambda=0.0735294, silent=-1,
               subsample=0.8715623, verbose=-1)

In [71]:
import pickle
pickle.dump(clf, open('saved_model.pkl', 'wb'))

In [72]:
fpr, tpr, _ = roc_curve(y_train, clf.predict_proba(x_train.values)[:, 1])
roc_auc = auc(fpr, tpr)

In [73]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

pred_val = clf.predict(x_train)
tp, fn, fp, tn = confusion_matrix(y_train, pred_val, labels=[1,0]).ravel()

In [74]:
conf_matrix = pd.DataFrame(
    confusion_matrix(y_train.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(y_train.values.ravel(), pred_val))

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("3. Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

1. Counfusion Matrix
                   True Value 0  True Value 1
Predicted Value 0        271476          8471
Predicted Value 1         26841        289846

2. Classification Report
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94    298317
         1.0       0.92      0.97      0.94    298317

    accuracy                           0.94    596634
   macro avg       0.94      0.94      0.94    596634
weighted avg       0.94      0.94      0.94    596634

3. Model Metric Sumamry
 - Accuracy Rate    : 94.081 %
 - Recall Rate      : 97.160 %
 - Precision Rate   : 91.524 %
 - Specificity Rate : 91.003 %
 - F1 Score         : 94.258 
 - ROC AUC          : 98.621 


# Try

In [ ]:
# AGGREGATIONS

BUREAU_AGG = {
    'SK_ID_BUREAU': ['nunique'],
    'DAYS_CREDIT': ['min', 'max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_ANNUITY': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    # Categorical
    'STATUS_0': ['mean'],
    'STATUS_1': ['mean'],
    'STATUS_12345': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed': ['mean'],
    'CREDIT_ACTIVE_Sold': ['mean'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],
    # Group by loan duration features (months)
    'LL_AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'LL_DEBT_CREDIT_DIFF': ['mean'],
    'LL_STATUS_12345': ['mean'],
}

BUREAU_ACTIVE_AGG = {
    'DAYS_CREDIT': ['max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['min', 'mean'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
}

BUREAU_CLOSED_AGG = {
    'DAYS_CREDIT': ['max', 'var'],
    'DAYS_CREDIT_ENDDATE': ['max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'sum'],
    'DAYS_CREDIT_UPDATE': ['max'],
    'ENDDATE_DIF': ['mean'],
    'STATUS_12345': ['mean'],
}

BUREAU_LOAN_TYPE_AGG = {
    'DAYS_CREDIT': ['mean', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DAYS_CREDIT_ENDDATE': ['max'],
}

BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'STATUS_0': ['mean'],
    'STATUS_12345': ['mean'],
}

PREVIOUS_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['max'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean', 'var'],
    'DOWN_PAYMENT_TO_CREDIT': ['mean'],
}

PREVIOUS_ACTIVE_AGG = {
    'SK_ID_PREV': ['nunique'],
    'SIMPLE_INTERESTS': ['mean'],
    'AMT_ANNUITY': ['max', 'sum'],
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['sum'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'AMT_PAYMENT': ['sum'],
    'INSTALMENT_PAYMENT_DIFF': ['mean', 'max'],
    'REMAINING_DEBT': ['max', 'mean', 'sum'],
    'REPAYMENT_RATIO': ['mean'],
}

PREVIOUS_APPROVED_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max'],
    'AMT_GOODS_PRICE': ['max'],
    'HOUR_APPR_PROCESS_START': ['min', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['mean'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['max'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    # The following features are only for approved applications
    'DAYS_FIRST_DRAWING': ['max', 'mean'],
    'DAYS_FIRST_DUE': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE': ['max', 'mean'],
    'DAYS_LAST_DUE_DIFF': ['min', 'max', 'mean'],
    'SIMPLE_INTERESTS': ['min', 'max', 'mean'],
}

PREVIOUS_REFUSED_AGG = {
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['min', 'max'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LATE_PAYMENTS_AGG = {
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LOAN_TYPE_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['min', 'mean', 'max', 'var'],
    'APPLICATION_CREDIT_DIFF': ['min', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['max'],
    'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean'],
    'CNT_PAYMENT': ['mean'],
}

PREVIOUS_TIME_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['mean', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

POS_CASH_AGG = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'SK_DPD': ['max', 'mean', 'sum', 'var'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
    'LATE_PAYMENT': ['mean']
}

INSTALLMENTS_AGG = {
    'SK_ID_PREV': ['size', 'nunique'],
    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],
    'LATE_PAYMENT': ['mean', 'sum'],
    'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
    'LATE_PAYMENT_RATIO': ['mean'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],
    'PAID_OVER': ['mean']
}

INSTALLMENTS_TIME_AGG = {
    'SK_ID_PREV': ['size'],
    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],
    'LATE_PAYMENT': ['mean'],
    'SIGNIFICANT_LATE_PAYMENT': ['mean'],
    'LATE_PAYMENT_RATIO': ['mean'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],
}

CREDIT_CARD_AGG = {
    'MONTHS_BALANCE': ['min'],
    'AMT_BALANCE': ['max'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['max'],
    'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
    'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
    'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'mean', 'sum', 'var'],
    'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
    'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_POS_CURRENT': ['mean'],
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['max', 'sum'],
    'LIMIT_USE': ['max', 'mean'],
    'PAYMENT_DIV_MIN': ['min', 'mean'],
    'LATE_PAYMENT': ['max', 'sum'],
}

CREDIT_CARD_TIME_AGG = {
    'CNT_DRAWINGS_ATM_CURRENT': ['mean'],
    'SK_DPD': ['max', 'sum'],
    'AMT_BALANCE': ['mean', 'max'],
    'LIMIT_USE': ['max', 'mean']
}

def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(path, nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols + ['MONTHS_BALANCE']].mean().reset_index()
    
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

def get_bureau(path, num_rows= None):
    """ Process bureau.csv and bureau_balance.csv and return a pandas dataframe. """
    bureau = pd.read_csv(path, nrows= num_rows)
    
    # Credit duration and credit/account end date difference
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    
    # Credit to debt ratio and difference
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

    # One-hot encoder
    bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category= False)
    
    # Join bureau balance features
    bureau = bureau.merge(get_bureau_balance(bureau_balance, num_rows), how='left', on='SK_ID_BUREAU')
    
    # Flag months with late payments (days past due)
    bureau['STATUS_12345'] = 0
    for i in range(1,6):
        bureau['STATUS_12345'] += bureau['STATUS_{}'.format(i)]

    # Aggregate by number of months in balance and merge with bureau (loan length agg)
    features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
        'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']
    agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean().reset_index()
    agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)
    bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')
    del agg_length; gc.collect()

    # General loans aggregations
    agg_bureau = group(bureau, 'BUREAU_', BUREAU_AGG)
    
    # Active and closed loans aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    agg_bureau = group_and_merge(active,agg_bureau,'BUREAU_ACTIVE_',BUREAU_ACTIVE_AGG)
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    agg_bureau = group_and_merge(closed,agg_bureau,'BUREAU_CLOSED_',BUREAU_CLOSED_AGG)
    del active, closed; gc.collect()
    
    # Aggregations for the main loan types
    for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
        type_df = bureau[bureau['CREDIT_TYPE_' + credit_type] == 1]
        prefix = 'BUREAU_' + credit_type.split(' ')[0].upper() + '_'
        agg_bureau = group_and_merge(type_df, agg_bureau, prefix, BUREAU_LOAN_TYPE_AGG)
        del type_df; gc.collect()
    
    # Time based aggregations: last x months
    for time_frame in [6, 12]:
        prefix = "BUREAU_LAST{}M_".format(time_frame)
        time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -30*time_frame]
        agg_bureau = group_and_merge(time_frame_df, agg_bureau, prefix, BUREAU_TIME_AGG)
        del time_frame_df; gc.collect()

    # Last loan max overdue
    sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
    gr = sort_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
    gr.rename({'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
    agg_bureau = agg_bureau.merge(gr, on='SK_ID_CURR', how='left')
    
    # Ratios: total debt/total credit and active loans debt/ active loans credit
    agg_bureau['BUREAU_DEBT_OVER_CREDIT'] = \
        agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
    agg_bureau['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = \
        agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']
    return agg_bureau

In [ ]:
get_bureau_balance(bureau_balance)

In [ ]:
get_bureau(bureau)